In [5]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import json
import os

def fetch_chroma_data(
    persist_directory="./chroma_db",
    collection_name="cv_sections",   # or "job_descriptions"
    limit=30
):
    """
    Fetch and display documents, metadata, and embeddings from a Chroma collection.

    Args:
        persist_directory (str): Path where ChromaDB was persisted.
        collection_name (str): Name of the collection to inspect.
        limit (int): Number of records to fetch.
    """

    print(f"📦 Loading Chroma collection: '{collection_name}' from {persist_directory}")

    # Recreate embeddings function (required by Chroma)
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")

    # Connect to existing Chroma collection
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name=collection_name
    )

    # Fetch all records
    try:
        data = vectorstore.get(limit=limit)
    except Exception as e:
        print(f"❌ Failed to fetch from Chroma: {e}")
        return

    # Display basic info
    num_docs = len(data.get("documents", []))
    print(f"✅ Retrieved {num_docs} records from '{collection_name}'\n")

    # Print a few sample records
    for i in range(min(num_docs, limit)):
        print(f"--- Record {i+1} ---")
        print("🆔 ID:", data["ids"][i])
        print("📄 Text:", data["documents"][i][:200], "..." if len(data["documents"][i]) > 200 else "")
        print("📑 Metadata:", json.dumps(data["metadatas"][i], indent=2))
        print()

    # Optional: return for programmatic use
    return data


if __name__ == "__main__":
    # Example usage
    # Fetch CV embeddings
    fetch_chroma_data(collection_name="cv_sections", limit=10)

    # Fetch JD embeddings
    fetch_chroma_data(collection_name="job_descriptions", limit=5)


📦 Loading Chroma collection: 'cv_sections' from ./chroma_db
✅ Retrieved 10 records from 'cv_sections'

--- Record 1 ---
🆔 ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484_email_0
📄 Text: aidooenochkwadwo@gmail.com 
📑 Metadata: {
  "chunk_id": 0,
  "embed_date": "2025-10-08T14:41:04.270518+00:00",
  "section": "email",
  "cv_id": "2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484"
}

--- Record 2 ---
🆔 ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484_summary_0
📄 Text: A Data Analyst with about two years of professional experience specialized in transforming complex datasets into strategic business solutions. I've consistently delivered actionable insights that have ...
📑 Metadata: {
  "embed_date": "2025-10-08T14:41:04.270518+00:00",
  "cv_id": "2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484",
  "chunk_id": 0,
  "section": "summary"
}

--- Record 3 ---
🆔 ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc

In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import json
import os

def fetch_chroma_data(
    persist_directory="./chroma_db",
    collection_name="cv_sections",
    limit=10
):
    """
    Fetch and display documents, metadata, and embeddings from a Chroma collection.
    """
    print(f"📦 Loading Chroma collection: '{collection_name}' from {persist_directory}")
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")

    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name=collection_name
    )

    try:
        # 1. FIX: Explicitly include embeddings
        data = vectorstore.get(
            limit=limit,
            include=['documents', 'metadatas', 'embeddings'] 
        )
    except Exception as e:
        print(f"❌ Failed to fetch from Chroma: {e}")
        return

    num_docs = len(data.get("documents", []))
    print(f"✅ Retrieved {num_docs} records from '{collection_name}'\n")

    for i in range(min(num_docs, limit)):
        print(f"--- Record {i+1} ---")
        print("🆔 ID:", data["ids"][i])
        print("📄 Text:", data["documents"][i][:200], "..." if len(data["documents"][i]) > 200 else "")
        print("📑 Metadata:", json.dumps(data["metadatas"][i], indent=2))
        
        embedding_vector = data.get("embeddings", [None]*num_docs)[i]
        
        # 2. FIX: Check length explicitly to avoid ValueError with NumPy arrays
        if embedding_vector is not None and len(embedding_vector) > 0:
            vector_size = len(embedding_vector)
            print(f"📐 Embedding Vector (Size {vector_size}): [ {embedding_vector[0]:.4f}, {embedding_vector[1]:.4f}, ..., {embedding_vector[-1]:.4f} ]")
        else:
            print("❌ Embedding Vector: NOT FOUND (Vector was None or empty)")

        print()

    return data


if __name__ == "__main__":
    # Example usage
    # Fetch CV embeddings
    fetch_chroma_data(collection_name="cv_sections", limit=36)
    # Fetch JD embeddings
    fetch_chroma_data(collection_name="job_descriptions", limit=5)

📦 Loading Chroma collection: 'cv_sections' from ./chroma_db
✅ Retrieved 36 records from 'cv_sections'

--- Record 1 ---
🆔 ID: 153f885ca3d2db24433ed0fdc60b67e72dcc4714b7c8860e4799b3eb3c45f3b8_email_0
📄 Text: aidooenoch@gmail.com 
📑 Metadata: {
  "section": "email",
  "chunk_id": 0,
  "cv_id": "153f885ca3d2db24433ed0fdc60b67e72dcc4714b7c8860e4799b3eb3c45f3b8",
  "embed_date": "2025-10-08T15:31:21.599399+00:00"
}
📐 Embedding Vector (Size 1024): [ -0.0004, 0.0005, ..., -0.0273 ]

--- Record 2 ---
🆔 ID: 153f885ca3d2db24433ed0fdc60b67e72dcc4714b7c8860e4799b3eb3c45f3b8_summary_0
📄 Text: A Data Analyst with about two years of professional experience specialized in transforming complex datasets into strategic business solutions. I've consistently delivered actionable insights that have ...
📑 Metadata: {
  "chunk_id": 0,
  "cv_id": "153f885ca3d2db24433ed0fdc60b67e72dcc4714b7c8860e4799b3eb3c45f3b8",
  "section": "summary",
  "embed_date": "2025-10-08T15:31:21.599399+00:00"
}
📐 Embedding Vector (S

In [13]:
fetch_chroma_data(collection_name="job_descriptions", limit=5)

📦 Loading Chroma collection: 'job_descriptions' from ./chroma_db
❌ Failed to fetch from Chroma: Error executing plan: Internal error: Error creating hnsw segment reader: Nothing found on disk


In [9]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import json
import os

def fetch_chroma_data(
    persist_directory="./jd_chroma_db",
    collection_name="cv_sections",
    limit=10
):
    """
    Fetch and display documents, metadata, and embeddings from a Chroma collection.
    """
    print(f"📦 Loading Chroma collection: '{collection_name}' from {persist_directory}")
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")

    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name=collection_name
    )

    try:
        # 1. FIX: Explicitly include embeddings
        data = vectorstore.get(
            limit=limit,
            include=['documents', 'metadatas', 'embeddings'] 
        )
    except Exception as e:
        print(f"❌ Failed to fetch from Chroma: {e}")
        return

    num_docs = len(data.get("documents", []))
    print(f"✅ Retrieved {num_docs} records from '{collection_name}'\n")

    for i in range(min(num_docs, limit)):
        print(f"--- Record {i+1} ---")
        print("🆔 ID:", data["ids"][i])
        print("📄 Text:", data["documents"][i][:200], "..." if len(data["documents"][i]) > 200 else "")
        print("📑 Metadata:", json.dumps(data["metadatas"][i], indent=2))
        
        embedding_vector = data.get("embeddings", [None]*num_docs)[i]
        
        # 2. FIX: Check length explicitly to avoid ValueError with NumPy arrays
        if embedding_vector is not None and len(embedding_vector) > 0:
            vector_size = len(embedding_vector)
            print(f"📐 Embedding Vector (Size {vector_size}): [ {embedding_vector[0]:.4f}, {embedding_vector[1]:.4f}, ..., {embedding_vector[-1]:.4f} ]")
        else:
            print("❌ Embedding Vector: NOT FOUND (Vector was None or empty)")

        print()

    return data


if __name__ == "__main__":
    # Example usage
    # Fetch JD embeddings
    fetch_chroma_data(collection_name="job_descriptions", limit=5)

📦 Loading Chroma collection: 'job_descriptions' from ./jd_chroma_db
✅ Retrieved 5 records from 'job_descriptions'

--- Record 1 ---
🆔 ID: f7ffdd206e7ee16b70acb2f0f00fbfd5d4f000766c9d02d286ca9e8dfa0f0486_job_title_0
📄 Text: Data Analyst 
📑 Metadata: {
  "jd_id": "f7ffdd206e7ee16b70acb2f0f00fbfd5d4f000766c9d02d286ca9e8dfa0f0486",
  "section": "job_title",
  "chunk_id": 0,
  "embed_date": "2025-10-08T14:02:09.709790+00:00"
}
📐 Embedding Vector (Size 1024): [ -0.0371, -0.0021, ..., 0.0158 ]

--- Record 2 ---
🆔 ID: f7ffdd206e7ee16b70acb2f0f00fbfd5d4f000766c9d02d286ca9e8dfa0f0486_required_skills_0
📄 Text: Advanced SQL proficiency (complex queries, joins, window functions, performance optimization) | Python or R for data manipulation, statistical analysis, and automation | Data visualization tools (Tabl ...
📑 Metadata: {
  "embed_date": "2025-10-08T14:02:09.710787+00:00",
  "section": "required_skills",
  "chunk_id": 0,
  "jd_id": "f7ffdd206e7ee16b70acb2f0f00fbfd5d4f000766c9d02d286ca9e8dfa0f0

In [12]:
import logging
import numpy as np
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from collections import defaultdict
from typing import Dict, List, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CVJDVectorSearch:
    """Performs vector search of CVs against a job description with section-wise scoring."""
    
    def __init__(
        self,
        cv_persist_dir="./chroma_db_cv",
        jd_persist_dir="./jd_chroma_db",
        cv_collection_name="cv_sections",
        jd_collection_name="job_descriptions",
        model="mxbai-embed-large",
        top_k_per_section=5
    ):
        """Initialize with Chroma collections and embedding model."""
        self.embeddings = OllamaEmbeddings(model=model)
        self.top_k_per_section = top_k_per_section
        
        # Initialize Chroma vector stores
        self.cv_vectorstore = Chroma(
            persist_directory=cv_persist_dir,
            embedding_function=self.embeddings,
            collection_name=cv_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )
        self.jd_vectorstore = Chroma(
            persist_directory=jd_persist_dir,
            embedding_function=self.embeddings,
            collection_name=jd_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )
        
        # Section mapping: JD section -> List of relevant CV sections
        self.section_mapping = {
            "job_title": ["summary"],
            "required_skills": ["skills", "work_experience"],
            "preferred_skills": ["skills", "work_experience"],
            "required_qualifications": ["education", "years_of_experience", "work_experience"],
            "education_requirements": ["education"],
            "experience_requirements": ["work_experience", "years_of_experience"],
            "technical_skills": ["skills"],
            "soft_skills": ["soft_skills"],
            "certifications": ["certifications"],
            "responsibilities": ["work_experience", "projects"]
        }
        
        # Section weights for scoring
        self.section_weights = {
            "required_skills": 0.3,
            "preferred_skills": 0.05,
            "required_qualifications": 0.05,
            "education_requirements": 0.05,
            "experience_requirements": 0.05,
            "technical_skills": 0.05,
            "soft_skills": 0.1,
            "certifications": 0.1,
            "responsibilities": 0.1,
            "job_title": 0.05
        }
        
    def fetch_jd_chunks(self) -> List[Dict]:
        """Fetch all chunks from the job_descriptions collection."""
        try:
            logger.info("Querying JD collection for all chunks")
            data = self.jd_vectorstore.get(
                include=["documents", "metadatas", "embeddings"]
            )
            chunks = [
                {
                    "text": data["documents"][i],
                    "metadata": data["metadatas"][i],
                    "embedding": data["embeddings"][i]
                }
                for i in range(len(data["documents"]))
            ]
            logger.info(f"Fetched {len(chunks)} chunks from job_descriptions collection")
            
            if len(chunks) == 0:
                all_data = self.jd_vectorstore.get(include=["metadatas"])
                all_jd_ids = set(meta.get("jd_id", "UNKNOWN") for meta in all_data.get("metadatas", []))
                logger.warning(f"No chunks found in job_descriptions. Available JD IDs: {all_jd_ids}")
            
            return chunks
        except Exception as e:
            logger.error(f"Failed to fetch JD chunks: {e}")
            return []
    
    def search_cv_chunks(
        self,
        jd_chunk_embedding: List[float],
        cv_sections: List[str],
        cv_id: str = None
    ) -> List[Dict]:
        """Search CV chunks matching a JD chunk embedding, filtered by CV sections and optional CV ID."""
        try:
            # Access the underlying Chroma client
            collection = self.cv_vectorstore._collection
            # Build where clause with $and operator if cv_id is provided
            if cv_id:
                where_clause = {
                    "$and": [
                        {"section": {"$in": cv_sections}},
                        {"cv_id": cv_id}
                    ]
                }
            else:
                where_clause = {"section": {"$in": cv_sections}}
            
            # Perform query using precomputed embedding
            results = collection.query(
                query_embeddings=[jd_chunk_embedding],
                n_results=self.top_k_per_section,
                where=where_clause,
                include=["documents", "metadatas", "distances"]
            )
            
            # Check if results are empty
            if not results["documents"] or not results["documents"][0]:
                logger.warning(f"No CV chunks found for sections {cv_sections} and cv_id {cv_id}")
                return []
            
            return [
                {
                    "text": results["documents"][0][i],
                    "metadata": results["metadatas"][0][i],
                    "score": results["distances"][0][i],
                    "cv_id": results["metadatas"][0][i]["cv_id"],
                    "section": results["metadatas"][0][i]["section"]
                }
                for i in range(len(results["documents"][0]))
            ]
        except Exception as e:
            logger.error(f"Error searching CV chunks: {e}")
            return []
    
    def compute_section_score(
        self,
        jd_section: str,
        jd_chunks: List[Dict],
        cv_id: str
    ) -> Tuple[float, List[Dict]]:
        """Compute similarity score for a JD section against a CV, handling multiple chunks."""
        cv_sections = self.section_mapping.get(jd_section, [])
        if not cv_sections:
            return 0.0, []
        
        similarities = []
        matched_chunks = []
        
        for jd_chunk in jd_chunks:
            if jd_chunk["metadata"]["section"] != jd_section:
                continue
            results = self.search_cv_chunks(
                jd_chunk_embedding=jd_chunk["embedding"],
                cv_sections=cv_sections,
                cv_id=cv_id
            )
            for result in results:
                # Convert distance (0 to 2, where 0 is identical) to similarity (0 to 1)
                similarity = 1 - (result["score"] / 2)
                similarities.append(similarity)
                matched_chunks.append({
                    "jd_chunk": jd_chunk["text"],
                    "cv_chunk": result["text"],
                    "cv_section": result["section"],
                    "similarity": similarity
                })
        
        # Average similarity across all matches for this section
        section_score = np.mean(similarities) if similarities else 0.0
        return section_score, matched_chunks
    
    def search_and_score_cvs(self, top_k_cvs: int = 5) -> List[Dict]:
        """Search and score CVs against the JD in job_descriptions collection."""
        # Fetch JD chunks
        jd_chunks = self.fetch_jd_chunks()
        if not jd_chunks:
            logger.error("No JD chunks found, aborting search")
            return []
        
        # Get all unique CV IDs
        cv_ids = set(self.cv_vectorstore.get()["metadatas"][i]["cv_id"] 
                    for i in range(len(self.cv_vectorstore.get()["metadatas"])))
        
        # Score each CV
        cv_scores = []
        for cv_id in cv_ids:
            section_scores = {}
            section_details = {}
            total_score = 0.0
            matched_sections = 0
            
            # Compute score for each JD section
            for jd_section in self.section_mapping.keys():
                score, matched_chunks = self.compute_section_score(jd_section, jd_chunks, cv_id)
                if score > 0:
                    section_scores[jd_section] = score
                    section_details[jd_section] = matched_chunks
                    total_score += score * self.section_weights.get(jd_section, 0.05)
                    matched_sections += 1
            
            # Normalize total score by number of matched sections
            if matched_sections > 0:
                total_score /= matched_sections
            else:
                total_score = 0.0
            
            cv_scores.append({
                "cv_id": cv_id,
                "total_score": total_score,
                "section_scores": section_scores,
                "section_details": section_details
            })
        
        # Sort CVs by total score
        cv_scores.sort(key=lambda x: x["total_score"], reverse=True)
        logger.info(f"Ranked {len(cv_scores)} CVs from job_descriptions collection")
        return cv_scores[:top_k_cvs]
    
    def print_results(self, results: List[Dict]):
        """Print ranked CVs with section-wise scores and details."""
        for i, result in enumerate(results):
            print(f"\n--- CV {i+1} (ID: {result['cv_id']}) ---")
            print(f"Total Score: {result['total_score']:.4f}")
            print("Section Scores:")
            for section, score in result["section_scores"].items():
                print(f"  {section}: {score:.4f}")
            print("Section Details:")
            for section, matches in result["section_details"].items():
                print(f"  {section}:")
                for match in matches:
                    print(f"    JD Chunk: {match['jd_chunk'][:100]}...")
                    print(f"    CV Chunk: {match['cv_chunk'][:100]}... (Section: {match['cv_section']})")
                    print(f"    Similarity: {match['similarity']:.4f}")
            print()

# Example usage
if __name__ == "__main__":
    searcher = CVJDVectorSearch(
        cv_persist_dir="./chroma_db",
        jd_persist_dir="./jd_chroma_db",
        cv_collection_name="cv_sections",
        jd_collection_name="job_descriptions",
        model="mxbai-embed-large",
        top_k_per_section=5
    )
    results = searcher.search_and_score_cvs(top_k_cvs=5)
    searcher.print_results(results)

2025-10-08 16:19:45,509 - INFO - Querying JD collection for all chunks
2025-10-08 16:19:45,518 - INFO - Fetched 12 chunks from job_descriptions collection
2025-10-08 16:19:45,773 - INFO - Ranked 3 CVs from job_descriptions collection



--- CV 1 (ID: 2b95c553fbd50733f96df36baaabea5e2c35098b8a16d270c2e4d5d2806f067e) ---
Total Score: 0.0734
Section Scores:
  job_title: 0.8868
  required_skills: 0.8122
  preferred_skills: 0.8213
  required_qualifications: 0.7875
  education_requirements: 0.8493
  experience_requirements: 0.7872
  technical_skills: 0.9298
  soft_skills: 0.8232
  responsibilities: 0.8187
Section Details:
  job_title:
    JD Chunk: Data Analyst...
    CV Chunk: A Data Analyst with about two years of professional experience specialized in transforming complex d... (Section: summary)
    Similarity: 0.8868
  required_skills:
    JD Chunk: Advanced SQL proficiency (complex queries, joins, window functions, performance optimization) | Pyth...
    CV Chunk: Python | R | AWS | Microsoft Excel | Google Sheets | Power BI | SQL | NumPy | Pandas | Scikit-Learn ... (Section: skills)
    Similarity: 0.8771
    JD Chunk: Advanced SQL proficiency (complex queries, joins, window functions, performance optimization) | Pyt

In [2]:
import logging
import numpy as np
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from collections import defaultdict
from typing import Dict, List, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CVJDVectorSearch:
    """Performs vector search of CVs against a job description with section-wise scoring."""
    
    def __init__(
        self,
        cv_persist_dir="./chroma_db_cv",
        jd_persist_dir="./jd_chroma_db",
        cv_collection_name="cv_sections",
        jd_collection_name="job_descriptions",
        model="mxbai-embed-large",
        top_k_per_section=5
    ):
        """Initialize with Chroma collections and embedding model."""
        self.embeddings = OllamaEmbeddings(model=model)
        self.top_k_per_section = top_k_per_section
        
        # Initialize Chroma vector stores
        self.cv_vectorstore = Chroma(
            persist_directory=cv_persist_dir,
            embedding_function=self.embeddings,
            collection_name=cv_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )
        self.jd_vectorstore = Chroma(
            persist_directory=jd_persist_dir,
            embedding_function=self.embeddings,
            collection_name=jd_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )
        
        # Section mapping: JD section -> List of relevant CV sections
        self.section_mapping = {
            "job_title": ["summary"],
            "required_skills": ["skills", "work_experience"],
            "preferred_skills": ["skills", "work_experience"],
            "required_qualifications": ["education", "years_of_experience", "work_experience"],
            "education_requirements": ["education"],
            "experience_requirements": ["work_experience", "years_of_experience"],
            "technical_skills": ["skills"],
            "soft_skills": ["soft_skills"],
            "certifications": ["certifications"],
            "responsibilities": ["work_experience", "projects"]
        }
        
        # Section weights for scoring
        self.section_weights = {
            "required_skills": 0.3,
            "preferred_skills": 0.05,
            "required_qualifications": 0.05,
            "education_requirements": 0.05,
            "experience_requirements": 0.05,
            "technical_skills": 0.05,
            "soft_skills": 0.1,
            "certifications": 0.1,
            "responsibilities": 0.1,
            "job_title": 0.05
        }
        
    def fetch_jd_chunks(self) -> List[Dict]:
        """Fetch all chunks from the job_descriptions collection."""
        try:
            logger.info("Querying JD collection for all chunks")
            data = self.jd_vectorstore.get(
                include=["documents", "metadatas", "embeddings"]
            )
            chunks = [
                {
                    "text": data["documents"][i],
                    "metadata": data["metadatas"][i],
                    "embedding": data["embeddings"][i]
                }
                for i in range(len(data["documents"]))
            ]
            logger.info(f"Fetched {len(chunks)} chunks from job_descriptions collection")
            
            if len(chunks) == 0:
                all_data = self.jd_vectorstore.get(include=["metadatas"])
                all_jd_ids = set(meta.get("jd_id", "UNKNOWN") for meta in all_data.get("metadatas", []))
                logger.warning(f"No chunks found in job_descriptions. Available JD IDs: {all_jd_ids}")
            
            return chunks
        except Exception as e:
            logger.error(f"Failed to fetch JD chunks: {e}")
            return []
    
    def search_cv_chunks(
        self,
        jd_chunk_embedding: List[float],
        cv_sections: List[str],
        cv_id: str = None
    ) -> List[Dict]:
        """Search CV chunks matching a JD chunk embedding, filtered by CV sections and optional CV ID."""
        try:
            # Access the underlying Chroma client
            collection = self.cv_vectorstore._collection
            # Build where clause with $and operator if cv_id is provided
            if cv_id:
                where_clause = {
                    "$and": [
                        {"section": {"$in": cv_sections}},
                        {"cv_id": cv_id}
                    ]
                }
            else:
                where_clause = {"section": {"$in": cv_sections}}
            
            # Perform query using precomputed embedding
            results = collection.query(
                query_embeddings=[jd_chunk_embedding],
                n_results=self.top_k_per_section,
                where=where_clause,
                include=["documents", "metadatas", "distances"]
            )
            
            # Check if results are empty
            if not results["documents"] or not results["documents"][0]:
                logger.warning(f"No CV chunks found for sections {cv_sections} and cv_id {cv_id}")
                return []
            
            return [
                {
                    "text": results["documents"][0][i],
                    "metadata": results["metadatas"][0][i],
                    "score": results["distances"][0][i],
                    "cv_id": results["metadatas"][0][i]["cv_id"],
                    "section": results["metadatas"][0][i]["section"]
                }
                for i in range(len(results["documents"][0]))
            ]
        except Exception as e:
            logger.error(f"Error searching CV chunks: {e}")
            return []
    
    def compute_section_score(
        self,
        jd_section: str,
        jd_chunks: List[Dict],
        cv_id: str
    ) -> Tuple[float, List[Dict]]:
        """Compute similarity score for a JD section against a CV, handling multiple chunks."""
        cv_sections = self.section_mapping.get(jd_section, [])
        if not cv_sections:
            return 0.0, []
        
        similarities = []
        matched_chunks = []
        
        for jd_chunk in jd_chunks:
            if jd_chunk["metadata"]["section"] != jd_section:
                continue
            results = self.search_cv_chunks(
                jd_chunk_embedding=jd_chunk["embedding"],
                cv_sections=cv_sections,
                cv_id=cv_id
            )
            for result in results:
                # Convert distance (0 to 2, where 0 is identical) to similarity (0 to 1)
                similarity = 1 - (result["score"] / 2)
                similarities.append(similarity)
                matched_chunks.append({
                    "jd_chunk": jd_chunk["text"],
                    "cv_chunk": result["text"],
                    "cv_section": result["section"],
                    "similarity": similarity
                })
        
        # Average similarity across all matches for this section
        section_score = np.mean(similarities) if similarities else 0.0
        return section_score, matched_chunks
    
    def search_and_score_cvs(self, top_k_cvs: int = 5) -> List[Dict]:
        """Search and score CVs against the JD in job_descriptions collection."""
        # Fetch JD chunks
        jd_chunks = self.fetch_jd_chunks()
        if not jd_chunks:
            logger.error("No JD chunks found, aborting search")
            return []
        
        # Get all unique CV IDs
        cv_ids = set(self.cv_vectorstore.get()["metadatas"][i]["cv_id"] 
                    for i in range(len(self.cv_vectorstore.get()["metadatas"])))
        
        # Score each CV
        cv_scores = []
        for cv_id in cv_ids:
            section_scores = {}
            section_details = {}
            total_score = 0.0
            total_weight = 0.0
            
            # Compute score for each JD section
            for jd_section in self.section_mapping.keys():
                score, matched_chunks = self.compute_section_score(jd_section, jd_chunks, cv_id)
                section_scores[jd_section] = score
                if score > 0:
                    section_details[jd_section] = matched_chunks
                total_score += score * self.section_weights.get(jd_section, 0.05)
                total_weight += self.section_weights.get(jd_section, 0.05)
            
            # Normalize total score by sum of weights
            if total_weight > 0:
                total_score /= total_weight
            else:
                total_score = 0.0
            
            cv_scores.append({
                "cv_id": cv_id,
                "total_score": total_score,
                "section_scores": section_scores,
                "section_details": section_details
            })
        
        # Sort CVs by total score
        cv_scores.sort(key=lambda x: x["total_score"], reverse=True)
        logger.info(f"Ranked {len(cv_scores)} CVs from job_descriptions collection")
        return cv_scores[:top_k_cvs]
    
    def print_results(self, results: List[Dict]):
        """Print ranked CVs with section-wise scores and details."""
        for i, result in enumerate(results):
            print(f"\n--- CV {i+1} (ID: {result['cv_id']}) ---")
            print(f"Total Score: {result['total_score']:.4f}")
            print("Section Scores:")
            for section, score in result["section_scores"].items():
                print(f"  {section}: {score:.4f}")
            print("Section Details:")
            for section, matches in result["section_details"].items():
                print(f"  {section}:")
                for match in matches:
                    print(f"    JD Chunk: {match['jd_chunk'][:100]}...")
                    print(f"    CV Chunk: {match['cv_chunk'][:100]}... (Section: {match['cv_section']})")
                    print(f"    Similarity: {match['similarity']:.4f}")
            print()

# Example usage
if __name__ == "__main__":
    searcher = CVJDVectorSearch(
        cv_persist_dir="./chroma_db",
        jd_persist_dir="./jd_chroma_db",
        cv_collection_name="cv_sections",
        jd_collection_name="job_descriptions",
        model="mxbai-embed-large",
        top_k_per_section=5
    )
    results = searcher.search_and_score_cvs(top_k_cvs=5)
    searcher.print_results(results)

2025-10-09 11:05:51,945 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-10-09 11:05:52,179 - INFO - Querying JD collection for all chunks
2025-10-09 11:05:52,191 - INFO - Fetched 12 chunks from job_descriptions collection
2025-10-09 11:05:52,695 - INFO - Ranked 3 CVs from job_descriptions collection



--- CV 1 (ID: 2b95c553fbd50733f96df36baaabea5e2c35098b8a16d270c2e4d5d2806f067e) ---
Total Score: 0.7344
Section Scores:
  job_title: 0.8868
  required_skills: 0.8122
  preferred_skills: 0.8213
  required_qualifications: 0.7875
  education_requirements: 0.8493
  experience_requirements: 0.7872
  technical_skills: 0.9298
  soft_skills: 0.8232
  certifications: 0.0000
  responsibilities: 0.8187
Section Details:
  job_title:
    JD Chunk: Data Analyst...
    CV Chunk: A Data Analyst with about two years of professional experience specialized in transforming complex d... (Section: summary)
    Similarity: 0.8868
  required_skills:
    JD Chunk: Advanced SQL proficiency (complex queries, joins, window functions, performance optimization) | Pyth...
    CV Chunk: Python | R | AWS | Microsoft Excel | Google Sheets | Power BI | SQL | NumPy | Pandas | Scikit-Learn ... (Section: skills)
    Similarity: 0.8771
    JD Chunk: Advanced SQL proficiency (complex queries, joins, window functions, perfor

In [4]:
import logging
import numpy as np
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from collections import defaultdict
from typing import Dict, List, Tuple, Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


class CVJDVectorSearch:
    """Performs vector search and scoring of CVs against a job description (JD) using embeddings and section-based logic."""

    def __init__(
        self,
        cv_persist_dir: str = "./chroma_db_cv",
        jd_persist_dir: str = "./jd_chroma_db",
        cv_collection_name: str = "cv_sections",
        jd_collection_name: str = "job_descriptions",
        model: str = "mxbai-embed-large",
        top_k_per_section: int = 5,
    ):
        """Initialize Chroma vector stores and embedding model."""
        self.embeddings = OllamaEmbeddings(model=model)
        self.top_k_per_section = top_k_per_section

        self.cv_vectorstore = Chroma(
            persist_directory=cv_persist_dir,
            embedding_function=self.embeddings,
            collection_name=cv_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )
        self.jd_vectorstore = Chroma(
            persist_directory=jd_persist_dir,
            embedding_function=self.embeddings,
            collection_name=jd_collection_name,
            collection_metadata={"hnsw:space": "cosine"}
        )

        # Section mapping: JD section → relevant CV sections
        self.section_mapping = {
            "job_title": ["summary"],
            "required_skills": ["skills", "work_experience"],
            "preferred_skills": ["skills", "work_experience"],
            "required_qualifications": ["education", "years_of_experience", "work_experience"],
            "education_requirements": ["education"],
            "experience_requirements": ["work_experience", "years_of_experience"],
            "technical_skills": ["skills"],
            "soft_skills": ["soft_skills"],
            "certifications": ["certifications"],
            "responsibilities": ["work_experience", "projects"],
        }

        # Section weights
        self.section_weights = {
            "required_skills": 0.3,
            "preferred_skills": 0.05,
            "required_qualifications": 0.05,
            "education_requirements": 0.05,
            "experience_requirements": 0.05,
            "technical_skills": 0.05,
            "soft_skills": 0.1,
            "certifications": 0.1,
            "responsibilities": 0.1,
            "job_title": 0.05,
        }

    # -------------------------------
    # 🔹 JD Fetching
    # -------------------------------
    def fetch_jd_chunks(self, jd_id: Optional[str] = None) -> List[Dict]:
        """Fetch all JD chunks (optionally filter by jd_id)."""
        try:
            logger.info("Fetching job description chunks...")
            data = self.jd_vectorstore.get(include=["documents", "metadatas", "embeddings"])
            chunks = [
                {
                    "text": data["documents"][i],
                    "metadata": data["metadatas"][i],
                    "embedding": data["embeddings"][i],
                }
                for i in range(len(data["documents"]))
            ]

            if jd_id:
                chunks = [c for c in chunks if c["metadata"].get("jd_id") == jd_id]

            logger.info(f"✅ Fetched {len(chunks)} JD chunks.")
            return chunks
        except Exception as e:
            logger.error(f"❌ Failed to fetch JD chunks: {e}")
            return []

    # -------------------------------
    # 🔹 CV Search Wrapper
    # -------------------------------
    def _safe_query_chroma(self, vectorstore: Chroma, **kwargs) -> Dict:
        """Safely query Chroma using public method or fallback to private collection."""
        try:
            return vectorstore.query(**kwargs)
        except Exception:
            logger.warning("Falling back to _collection.query() for Chroma compatibility.")
            collection = vectorstore._collection
            return collection.query(**kwargs)

    def search_cv_chunks(
        self,
        jd_chunk_embedding: List[float],
        cv_sections: List[str],
        cv_id: Optional[str] = None,
    ) -> List[Dict]:
        """Search CV chunks matching a JD chunk embedding, filtered by section and optionally CV ID."""
        try:
            where_clause = {"section": {"$in": cv_sections}}
            if cv_id:
                where_clause = {
                    "$and": [
                        {"section": {"$in": cv_sections}},
                        {"cv_id": cv_id}
                    ]
                }

            results = self._safe_query_chroma(
                self.cv_vectorstore,
                query_embeddings=[jd_chunk_embedding],
                n_results=self.top_k_per_section,
                where=where_clause,
                include=["documents", "metadatas", "distances"],
            )

            if not results.get("documents") or not results["documents"][0]:
                logger.debug(f"No results for CV sections {cv_sections} (cv_id={cv_id})")
                return []

            return [
                {
                    "text": results["documents"][0][i],
                    "metadata": results["metadatas"][0][i],
                    "score": results["distances"][0][i],
                    "cv_id": results["metadatas"][0][i]["cv_id"],
                    "section": results["metadatas"][0][i]["section"],
                }
                for i in range(len(results["documents"][0]))
            ]
        except Exception as e:
            logger.error(f"Error searching CV chunks: {e}")
            return []

    # -------------------------------
    # 🔹 Section Scoring
    # -------------------------------
    def compute_section_score(
        self, jd_section: str, jd_chunks: List[Dict], cv_id: str
    ) -> Tuple[float, List[Dict]]:
        """Compute similarity score for a JD section vs a CV."""
        cv_sections = self.section_mapping.get(jd_section, [])
        if not cv_sections:
            return 0.0, []

        similarities, matched_chunks = [], []

        for jd_chunk in jd_chunks:
            if jd_chunk["metadata"].get("section") != jd_section:
                continue

            results = self.search_cv_chunks(
                jd_chunk_embedding=jd_chunk["embedding"],
                cv_sections=cv_sections,
                cv_id=cv_id,
            )
            for r in results:
                similarity = 1 - (r["score"] / 2)  # Convert cosine distance → similarity
                similarities.append(similarity)
                matched_chunks.append({
                    "jd_chunk": jd_chunk["text"],
                    "cv_chunk": r["text"],
                    "cv_section": r["section"],
                    "similarity": similarity,
                })

        section_score = np.mean(similarities) if similarities else 0.0
        return section_score, matched_chunks

    # -------------------------------
    # 🔹 Main Scoring Function
    # -------------------------------
    def search_and_score_cvs(
        self, top_k_cvs: int = 5, jd_id: Optional[str] = None
    ) -> List[Dict]:
        """Search and score all CVs against a JD (optionally specify jd_id)."""
        jd_chunks = self.fetch_jd_chunks(jd_id)
        if not jd_chunks:
            logger.error("No JD chunks found; aborting search.")
            return []

        cv_data = self.cv_vectorstore.get(include=["metadatas"])
        cv_ids = {meta["cv_id"] for meta in cv_data["metadatas"]}
        logger.info(f"Found {len(cv_ids)} unique CVs to score.")

        cv_scores = []

        for cv_id in cv_ids:
            section_scores, section_details = {}, {}
            weighted_sum, total_weight = 0.0, 0.0

            for jd_section in self.section_mapping.keys():
                score, matches = self.compute_section_score(jd_section, jd_chunks, cv_id)
                section_scores[jd_section] = score
                if matches:
                    section_details[jd_section] = matches
                    weight = self.section_weights.get(jd_section, 0.05)
                    weighted_sum += score * weight
                    total_weight += weight

            total_score = (weighted_sum / total_weight) if total_weight > 0 else 0.0

            cv_scores.append({
                "cv_id": cv_id,
                "total_score": total_score,
                "section_scores": section_scores,
                "section_details": section_details,
            })

        cv_scores.sort(key=lambda x: x["total_score"], reverse=True)
        logger.info(f"✅ Ranked {len(cv_scores)} CVs successfully.")
        return cv_scores[:top_k_cvs]

    # -------------------------------
    # 🔹 Result Display
    # -------------------------------
    def print_results(self, results: List[Dict], show_details: bool = False):
        """Print ranked CVs with section-wise scores (and optional detail view)."""
        for i, result in enumerate(results):
            print(f"\n=== CV {i+1} | ID: {result['cv_id']} ===")
            print(f"Total Score: {result['total_score']:.4f}\n")
            print("Section Scores:")
            for section, score in result["section_scores"].items():
                print(f"  {section:25}: {score:.4f}")

            if show_details and result["section_details"]:
                print("\nDetailed Matches:")
                for section, matches in result["section_details"].items():
                    print(f"\n  [{section.upper()}]")
                    for match in matches[:2]:  # show top 2 per section
                        print(f"    JD: {match['jd_chunk'][:100]}...")
                        print(f"    CV: {match['cv_chunk'][:100]}...")
                        print(f"    Similarity: {match['similarity']:.4f}")
            print("-" * 60)


# Example usage
if __name__ == "__main__":
    searcher = CVJDVectorSearch(
        cv_persist_dir="./chroma_db",
        jd_persist_dir="./jd_chroma_db",
        cv_collection_name="cv_sections",
        jd_collection_name="job_descriptions",
        model="mxbai-embed-large",
        top_k_per_section=5,
    )
    results = searcher.search_and_score_cvs(top_k_cvs=1)
    searcher.print_results(results, show_details=True)


2025-10-09 11:51:26,558 - INFO - Fetching job description chunks...
2025-10-09 11:51:26,576 - INFO - ✅ Fetched 12 JD chunks.
2025-10-09 11:51:26,584 - INFO - Found 3 unique CVs to score.
2025-10-09 11:51:26,587 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,594 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,603 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,610 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,619 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,629 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,636 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,643 - WARNING - Falling back to _collection.query() for Chroma compatibility.
2025-10-09 11:51:26,649 - WAR


=== CV 1 | ID: 2b95c553fbd50733f96df36baaabea5e2c35098b8a16d270c2e4d5d2806f067e ===
Total Score: 0.8262

Section Scores:
  job_title                : 0.8868
  required_skills          : 0.8122
  preferred_skills         : 0.8213
  required_qualifications  : 0.7875
  education_requirements   : 0.8493
  experience_requirements  : 0.7872
  technical_skills         : 0.9298
  soft_skills              : 0.8232
  certifications           : 0.0000
  responsibilities         : 0.8187

Detailed Matches:

  [JOB_TITLE]
    JD: Data Analyst...
    CV: A Data Analyst with about two years of professional experience specialized in transforming complex d...
    Similarity: 0.8868

  [REQUIRED_SKILLS]
    JD: Advanced SQL proficiency (complex queries, joins, window functions, performance optimization) | Pyth...
    CV: Python | R | AWS | Microsoft Excel | Google Sheets | Power BI | SQL | NumPy | Pandas | Scikit-Learn ...
    Similarity: 0.8771
    JD: Advanced SQL proficiency (complex queries, joins,